# dicom2videos

In [3]:
import pydicom
import sys
import os
from pydicom.data import get_testdata_files
from typing import Any

def generateImagesFromDicom(dataset, indent=0, filename="demo"):
    """Go through all items in the dataset and print them with custom format

    Modelled after Dataset._pretty_str()
    """
    dont_print = ['Pixel Data', 'File Meta Information Version']
    dont_print = []

    indent_string = "   " * indent
    next_indent_string = "   " * (indent + 1)

    for data_element in dataset:
        if data_element.VR == "SQ":   # a sequence
            print(indent_string, data_element.name)
            for sequence_item in data_element.value:
                generateImagesFromDicom(dataset=sequence_item, indent=indent + 1,filename=filename)
                print(next_indent_string + "---------")
        else:
            if data_element.name in dont_print:
                print("""<item not printed -- in the "don't print" list>""")
            elif data_element.name=='Pixel Data' : 
                # check frame number and tranfersyntex here

                frmpath = filename+'_frms'
                os.makedirs(filename+'_frms', exist_ok=True)
                frmindex = 1
                print(indent_string, 'Extract frames from [Pixel Data]:')
                for frm in pydicom.encaps.generate_pixel_data_frame(data_element.value):
                
                    frmname = os.path.join(frmpath, "frm-%04d.png"%(frmindex,))
                    frmindex += 1
                    print("generateImagesFromDicom: %s %s : % 8d bytes"%(next_indent_string, frmname, len(frm)))

                    with open(frmname, 'wb+') as f:
                        f.write(frm)
            else:
                repr_value = repr(data_element.value)
                if len(repr_value) > 50:
                    repr_value = repr_value[:50] + "..."
                print("{0:s} {1:s} = {2:s}".format(indent_string,
                                                   data_element.name,
                                                   repr_value))
    if indent ==0:
        return frmpath


def pre_process_dicom(dicom_file):
    print("dcmfile:", dicom_file)
    if os.path.isfile(dicom_file):
        print("dcmfile is file? :", dicom_file)
        ds = pydicom.dcmread(dicom_file)
        image_dir = generateImagesFromDicom(ds, filename=dicom_file)
        return image_dir
    else:
        return ''

In [4]:
def dicoms2videos(dcm_root):
    if not os.path.exists(dcm_root):
        print("dicom path not correct")
        raise NotADirectoryError(dcm_root)
    dcm_files = []
    if os.path.isfile(dcm_root):
        dcm_files.append(dcm_root)
    elif os.path.isdir(dcm_root):
        for i in os.listdir(dcm_root):
            f = os.path.join(dcm_root, i)
            if '.dcm' in i and os.path.isfile(f):
                dcm_files.append(f)
    img_dirs = []
    print('dcm_files:', dcm_files)
    for df in dcm_files:
        img_dir = pre_process_dicom(dicom_file=df)
        img_dirs.append(img_dir)

In [6]:
docms_dir=r"C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2"
dicoms2videos(docms_dir)

dcm_files: ['C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.2.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.4.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.5.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.6.1.dcm', 'C:\\Users\\admin\\Desktop\\scanEmpty-SM-N9500.20190820\\1.3.6.1.4.1.52026.50937601.7313.2\\1.3.6.1.4.1.52026.50937601.7313.2.7.1.dcm']
dcmfile: C:\Users\admin\Desktop\scan

generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm_frms\frm-0035.png :   113794 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm_frms\frm-0036.png :   113256 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm_frms\frm-0037.png :   113182 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm_frms\frm-0038.png :   113258 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.1.1.dcm_frms\frm-0039.png :   113104 bytes
generateImagesFromDicom:     C:\Users\admin\D

 Group Length = 446
 Specific Character Set = 'ISO_IR 192'
 Image Type = ['ORIGINAL', 'PRIMARY', '', '0001']
 SOP Class UID = '1.2.840.10008.5.1.4.1.1.3.1'
 SOP Instance UID = '1.3.6.1.4.1.52026.50937601.7313.2.3.1'
 Study Date = '20190820'
 Series Date = '20190820'
 Content Date = '20190820'
 Study Time = '153354'
 Series Time = '153516'
 Content Time = '153525'
 Accession Number = 'A0001'
 Modality = 'US'
 Manufacturer = 'JiShi Medical'
 Institution Name = 'JiShiMed'
 Study Description = 'Nerve'
 Series Description = 'Nerve'
 Manufacturer's Model Name = 'Nerve'
 Derivation Description = 'Lossy compression with JPEG baseline, quality fac...
 Start Trim = "0"
 Stop Trim = "292"
 Recommended Display Frame Rate = "15"
 Group Length = 60
 Patient's Name = '某^人^^^'
 Patient ID = 'P0000002'
 Patient's Birth Date = '19731208'
 Patient's Sex = 'M'
 Group Length = 384
 Body Part Examined = 'Nerve'
 Cine Rate = "15"
 Software Version(s) = '100'
 Protocol Name = ''
 Frame Time = "66.6667"
 Outpu

generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm_frms\frm-0196.png :   114440 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm_frms\frm-0197.png :   112510 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm_frms\frm-0198.png :   112222 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm_frms\frm-0199.png :   113348 bytes
generateImagesFromDicom:     C:\Users\admin\Desktop\scanEmpty-SM-N9500.20190820\1.3.6.1.4.1.52026.50937601.7313.2\1.3.6.1.4.1.52026.50937601.7313.2.3.1.dcm_frms\frm-0200.png :   114060 bytes
generateImagesFromDicom:     C:\Users\admin\D

 Group Length = 444
 Specific Character Set = 'ISO_IR 192'
 Image Type = ['ORIGINAL', 'PRIMARY', '', '0001']
 SOP Class UID = '1.2.840.10008.5.1.4.1.1.3.1'
 SOP Instance UID = '1.3.6.1.4.1.52026.50937601.7313.2.7.1'
 Study Date = '20190820'
 Series Date = '20190820'
 Content Date = '20190820'
 Study Time = '153354'
 Series Time = '153627'
 Content Time = '153629'
 Accession Number = 'A0001'
 Modality = 'US'
 Manufacturer = 'JiShi Medical'
 Institution Name = 'JiShiMed'
 Study Description = 'Nerve'
 Series Description = 'Nerve'
 Manufacturer's Model Name = 'Nerve'
 Derivation Description = 'Lossy compression with JPEG baseline, quality fac...
 Start Trim = "0"
 Stop Trim = "89"
 Recommended Display Frame Rate = "15"
 Group Length = 60
 Patient's Name = '某^人^^^'
 Patient ID = 'P0000002'
 Patient's Birth Date = '19731208'
 Patient's Sex = 'M'
 Group Length = 384
 Body Part Examined = 'Nerve'
 Cine Rate = "15"
 Software Version(s) = '100'
 Protocol Name = ''
 Frame Time = "66.6667"
 Output